# _Cassia_ data re-analysis

Analysis of _Cassia_ data taken at 1 K.
This data was taken for three values of still heating power: 0, 30, 110 mW.
We only use the 0 mW data for the actual paper.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import copy
import itertools
import pickle
from pathlib import Path

import h5py
import matplotlib.pyplot as plt
import numpy as np
import skrf as rf
import uncertainties
from scipy.constants import c, pi
from uncertainties import ufloat
from uncertainties import unumpy as unp

In [3]:
from suprtools.fp_theory.geometry import SymmetricCavityGeometry
# from suprtools.fp_theory.coupling import CouplingConfig
# from suprtools.fp_theory.modes import ScalarModeBasis
from suprtools.plotting import sslab_style
from suprtools.rf import WideScanData, Ringdown, fit_mode, test_a_fit, FabryPerotModeParams, ModeMaskingPolicy

In [4]:
mirror_test_dir = Path('../data')
caecilia_data_dir = mirror_test_dir / 'Caecilia'
cassia_data_dir = mirror_test_dir / 'Cassia'
domitia_data_dir = mirror_test_dir / 'Domitia'
egnatia_data_dir = mirror_test_dir / 'Egnatia'

In [5]:
def load_all_windows(directory, name: str):
    dirpath = Path(directory)
    return tuple(
        WideScanData.from_window(h5_path, name, sample_temps=True)
        for h5_path in sorted(list(dirpath.glob('*.h5')))
    )

In [6]:
narrow_scan_dir = cassia_data_dir / '20230721-narrow-sweeps'
narrow_30mw_dir = cassia_data_dir / '20230724-narrow-sweeps-30mw'
narrow_110mw_dir = cassia_data_dir / '20230726-narrow-sweeps-110mw-turbo-off'

In [7]:
vna_data_0mw = load_all_windows(narrow_scan_dir, '7/21 narrow scans')
vna_data_30mw = load_all_windows(narrow_30mw_dir, '7/24 30 mW scans')
vna_data_110mw = load_all_windows(narrow_110mw_dir, '7/26 110 mW scans')
vna_data = [vna_data_0mw, vna_data_30mw, vna_data_110mw]

In [8]:
# shape: 4 (mode series) x 2 (polarizations) x 10 (FSRs)
fit_params_1k = [
    [
        [
            (77.11230, 5, 3),
            (80.53896, 5, 3),
            (83.96562, 5, 3),
            (87.39240, 5, 3),
            (90.81805, 10, 5),
            (94.24486, 0.8, 2),
            (97.67145, 7, 3),
            (101.09796, 0.4, 2),
            (104.52444, 0.4, 2),
            (107.95090, 0.4, 2),
        ],
        [
            (77.11286, 5, 4),
            (80.53950, 5, 3),
            (83.96614, 5, 3),
            (87.39287, 5, 3),
            (90.81851, 10, 5),
            (94.24533, 0.5, 2),
            (97.67189, 7, 3),
            (101.09839, 0.4, 2),
            (104.52487, 0.4, 2),
            (107.95131, 0.4, 2),
        ],
    ],
    [
        [
            None,
            None,
            None,
            None,
            (90.82038, 7, 5),
            None,
            None,
            None,
            None,
            None,
        ],
        [
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
        ],
    ],
    [
        [
            (77.11684, 8, 3),
            (80.54500, 8, 2),
            (83.97270, 8, 3),
            (87.40035, 12, 3),
            (90.82792, 6, 3),
            (94.25535, 6, 3),
            (97.68269, 5, 4),
            (101.10995, 5, 3),
            (104.53710, 5, 3),
            (107.96424, 5, 3),
        ],
        [
            (77.11696, 8, 3),
            None,
            (83.97285, 8, 3),
            (87.40057, 12, 3),
            (90.82812, 6, 3),
            (94.25554, 6, 3),
            (97.68289, 5, 4),
            (101.11014, 5, 3),
            (104.53730, 5, 3),
            (107.96443, 5, 3),
        ],
    ],
    [
        [
            (77.17183, 6, 3),
            (80.59820, 4, 3),
            (84.02458, 4, 3),
            None,
            (90.87730, 6, 4),
            (94.30365, 6, 3),
            (97.73004, 6, 3),
            None,
            (104.58280, 6, 3),
            (108.00920, 5, 4),
        ],
        [
            (77.17266, 6, 3),
            (80.59890, 4, 3),
            (84.02528, 4, 3),
            None,
            (90.87787, 6, 4),
            (94.30421, 6, 3),
            (97.73056, 6, 3),
            None,
            (104.58329, 6, 3),
            (108.00965, 5, 4),
        ],
    ],
]

In [10]:
def t5_key(fit_data_tuple):
    # T_5 = still plate temperature
    pole, t5, tsamp = fit_data_tuple
    return t5

In [11]:
mode_fit_data = [
    [
        {
            mode_tuple[0]: []  # will be (pole, still_temp, sample_temp)
            for mode_tuple in mode_series if mode_tuple is not None
        }
        for mode_series in multiplet_series
    ]
    for multiplet_series in fit_params_1k  # for each transverse mode up to polarization
]

In [12]:
for vna_data_single in itertools.chain(*vna_data):
    for i, multiplet_series in enumerate(fit_params_1k):
        for j, mode_series in enumerate(multiplet_series):
            for fit_param_onemode in mode_series:
                network = vna_data_single.s21
                if fit_param_onemode is None:
                    continue
                freq_ghz, span_mhz, n_poles_cmplx = fit_param_onemode
                if network.frequency.start > freq_ghz * 1e9:
                    continue
                if network.frequency.stop < freq_ghz * 1e9:
                    break

                pole_record = (ufloat(np.nan, np.nan),) * 4
                try:
                    vf = fit_mode(network, freq_ghz, span_mhz/1e3, n_poles_cmplx=n_poles_cmplx)
                    pole_record = vf.closest_pole_uparams(1e+9 * freq_ghz)
                except RuntimeError:
                    pass
                
                mode_fit_data[i][j][freq_ghz].append((
                    pole_record,
                    vna_data_single.metadata['t_plate'].item(),
                    vna_data_single.metadata['t_samp'].item(),
                ))
                
        #         mode_series_vfs.append(vf)
        #     multiplet_series_poles.append(mode_series_poles)
        #     multiplet_series_vfs.append(mode_series_vfs)
        # all_poles.append(multiplet_series_poles)
        # vfs.append(multiplet_series_vfs)

for mode_series in mode_fit_data:
    for pol_series in mode_series:
        for fit_data_tuples in pol_series.values():
            fit_data_tuples.sort(key=t5_key)

/Users/txz/Dropbox (Personal)/stanford/sslab/suprtools/build/__editable__.suprtools-2025.8.14-py3-none-any/suprtools/rf/freqdomain.py:1520: RuntimeWarning: Vector Fitting: The pole relocation process stopped after reaching the maximum number of iterations (N_max = 100). The results did not converge properly.
Hint: the linear system was ill-conditioned (max. condition number was 6.624128335718089e+18).
  vf.vector_fit(n_poles_real=0, n_poles_cmplx=n_poles_cmplx)
/Users/txz/Dropbox (Personal)/stanford/sslab/suprtools/build/__editable__.suprtools-2025.8.14-py3-none-any/suprtools/rf/freqdomain.py:1520: RuntimeWarning: Vector Fitting: The pole relocation process stopped after reaching the maximum number of iterations (N_max = 100). The results did not converge properly.
Hint: the linear system was ill-conditioned (max. condition number was 3.7983077438103706e+18).
  vf.vector_fit(n_poles_real=0, n_poles_cmplx=n_poles_cmplx)
/Users/txz/Dropbox (Personal)/stanford/sslab/suprtools/build/__edit

In [13]:
mode_fit_data_1k = copy.deepcopy(mode_fit_data)
for mode_series in mode_fit_data_1k:
    for pol_series in mode_series:
        for key, fit_data_tuples in pol_series.items():
            fit_data_tuples.sort(key=t5_key)

In [27]:
len(mode_fit_data_1k), len(mode_fit_data_1k[0]), len(mode_fit_data_1k[0][0])

(4, 2, 10)

In [ ]:
# with open('cassia-mode-fit-arr.pickle', 'wb') as f:
#     pickle.dump(mode_fit_data_1k, f)

In [ ]:
# with open('cassia-mode-fit-arr.pickle', 'rb') as f:
#     mode_fit_data_1k = pickle.load(f)

In [16]:
cassia_fsr = 3.42621675e+9

In [51]:
import pandas as pd

cassia_00_mode_params_df = pd.DataFrame([
    {
        'mode_q': int(1e9 * freq_ghz / cassia_fsr),  # longitudinal index, NOT Q factor
        'pol': pol,
        'heater_power_mw': heater_power_mw,
        'pole_real_n': record[0].n,
        'pole_real_s': record[0].s,
        'pole_imag_n': record[1].n,
        'pole_imag_s': record[1].s,
        'residue_real_n': record[2].n,
        'residue_real_s': record[2].s,
        'residue_imag_n': record[3].n,
        'residue_imag_s': record[3].s,
        't_plate': t_plate,
        't_samp': t_samp,
    }
    for pol, mode_records in zip(['x', 'y'], mode_fit_data_1k[0])
    for freq_ghz, records in mode_records.items()
    for heater_power_mw, (record, t_plate, t_samp) in zip([0, 30, 110], records)
])
cassia_00_mode_params_df.set_index(['heater_power_mw', 'pol', 'mode_q'], inplace=True)
cassia_00_mode_params_df.sort_index(inplace=True)
cassia_00_mode_params_df_noheaterpower = cassia_00_mode_params_df.xs(0, level='heater_power_mw')

In [52]:
cassia_00_mode_params_df_noheaterpower

pole_real_n  pole_real_s   pole_imag_n  pole_imag_s  \
pol mode_q                                                          
x   22      -35639.261205   162.529274  4.845109e+11   161.735960   
    23      -41909.605528  1134.692882  5.060412e+11  1138.190947   
    24     -133873.743829   641.373441  5.275716e+11   641.363554   
    25     -504464.262550   606.494896  5.491027e+11   706.232172   
    26     -480008.141630  1044.782639  5.706266e+11  1087.490073   
    27     -101282.929198  1603.640847  5.921580e+11  1527.178561   
    28      -30102.371612   184.974773  6.136878e+11   183.806209   
    29      -16491.873000   576.917912  6.352172e+11   572.820596   
    30      -14131.752792   847.331372  6.567465e+11   845.634606   
    31      -12907.307539   804.925560  6.782755e+11   804.906096   
y   22      -49743.240584  1392.915199  4.845145e+11  1397.051217   
    23      -50407.396155   810.249275  5.060446e+11   821.468757   
    24      -76383.148071   171.209811  5.275749e+11   171.214829   
    25     -650024.238612  2946.930188  5.491056e+11  3735.612627   
    26     -884957.499518  5864.777437  5.706293e+11  5719.439838   
    27     -102844.490277  1360.479126  5.921608e+11  1442.843940   
    28      -32833.187837    80.958787  6.136906e+11    80.302647   
    29      -15932.720923   239.255542  6.352200e+11   236.302820   
    30       -8686.047782   242.944742  6.567491e+11   241.312056   
    31       -9022.445583   404.211980  6.782781e+11   407.069210   

            residue_real_n  residue_real_s  residue_imag_n  residue_imag_s  \
pol mode_q                                                                   
x   22            0.277660        0.007301       -2.181637        0.007330   
    23           -0.371850        0.008328       -0.201748        0.008561   
    24           -1.722973        0.008675       -1.510188        0.008679   
    25          -32.733090        0.091936       45.188476        0.094780   
    26            7.603601        0.040307      -14.504555        0.041627   
    27            0.339034        0.005738        0.308273        0.005695   
    28           -0.539885        0.003039       -0.402671        0.002978   
    29            0.062120        0.002044       -0.049149        0.001995   
    30           -0.021375        0.001367        0.022450        0.001338   
    31            0.012424        0.002336       -0.050357        0.002336   
y   22            0.316607        0.008444        0.249852        0.008840   
    23            0.702272        0.009332        0.357578        0.009702   
    24            0.318275        0.005939       -3.503157        0.005939   
    25           -9.229755        0.151120      -16.232936        0.117121   
    26            5.801175        0.058227        5.586130        0.065562   
    27            0.372806        0.008916       -0.675687        0.008753   
    28            1.529060        0.003134        0.859923        0.003149   
    29           -0.114748        0.002129       -0.159108        0.002149   
    30           -0.034646        0.001007        0.036300        0.001006   
    31            0.109338        0.003764       -0.041223        0.003805   

             t_plate    t_samp  
pol mode_q                      
x   22      0.705220  1.028999  
    23      0.701236  1.004638  
    24      0.715110  1.075969  
    25      0.718912  1.136177  
    26      0.720592  1.102364  
    27      0.731488  1.173603  
    28      0.728376  1.104254  
    29      0.713579  1.062879  
    30      0.713424  1.066240  
    31      0.728763  1.089646  
y   22      0.705220  1.028999  
    23      0.701236  1.004638  
    24      0.715110  1.075969  
    25      0.718912  1.136177  
    26      0.720592  1.102364  
    27      0.731488  1.173603  
    28      0.728376  1.104254  
    29      0.713579  1.062879  
    30      0.713424  1.066240  
    31      0.728763  1.089646

In [54]:
cassia_00_mode_params_df_noheaterpower.to_csv('cassia_00_mode_params.csv', na_rep='NaN')

In [ ]:
# with open('cassia-00-mode-params.pickle', 'wb') as f:
#     pickle.dump(cassia_00_mode_params, f)

In [53]:
# cassia_00_mode_params